In [4]:
import random
from SUMOSimulation import SUMO

# Ubicación de los archivos de simulación de la red vial:
infoSUMO = {
    'projectFolder': r"SUMO_Simulation\RedVialZMG",
    'netFile': "\osm.net.xml", # Nombre del archivo que contiene la red
    'sumoCfgFile': "\osm.sumocfg" # Nombre del archivo principal de ejecución de SUMO
    }

#Dicionario de clusters de intersecciones: {id_custer: [tiempos en verde de cada semáforo], [archivos de detectores de vehículos]}
ampelDict = {"cluster_1395059763_1658290098": [[40, 50, 60], ["\Torres.xml", "\Belizario.xml", "\Torres2Belizario.xml"]]} 

# Parámetros de inicio del algoritmo genético:
SIMULATION_TIME = 900 # Tiempo de ejecución de la simulación en segundos
POPULATION_SIZE = 20
GENERATIONS = 50
CROSSOVER_ODDS = .8
MUTATION_ODDS = .2
MIN_TIME = 20 # Tiempo mínimo en verde
MAX_TIME = 120 # Tiempo máximo en verde

nAmpels = len(ampelDict["cluster_1395059763_1658290098"][0])


def generatePopulation():
    """Se genera la población inicial de manera aleatoria"""
    return [[random.randint(MIN_TIME, MAX_TIME) for _ in range(nAmpels)] for _ in range(POPULATION_SIZE)]

def calculateFitness(chromosome, vehicleRates):
    """Se calcula el tiempo total y penaliza los semáforos con flujo más bajo"""
    #totalTime = sum(chromosome) # Mayor tiempo en verde indica mayor flujo de autos
    totalFlow = 0 # Suma de tasas de flujos
    penalization = 0 # Penalización de tiempo para el cromosoma

    for i in range(len(chromosome)):
        time = chromosome[i] # Tiempo del alelo actual (semáforo)
        cVehicleRate = vehicleRates[i] # Tasa de vehiculos correspondiente al alelo actual (semáforo)
        flow = time * cVehicleRate # Factor de flujo = tiempo en verde * la casa de vehiculos del semáforo actual
        totalFlow += flow

        # Se penalizan largos tiempos en verde para tasas bajas de vehículos:
        if cVehicleRate < 0.1:
            penalization += (time - MIN_TIME) * 2

        #Calcular el fitness del cromosoma:
        fitness = totalFlow - penalization
    
    return fitness

def selection(population, fitness):
    """Se selecciona de manera elitista el mejor individuo de la generación"""
    fittedPopulation = list(zip(population, fitness)) # Se crean pares (población, fitness)
    bestFit = max(fittedPopulation, key=lambda x: x[1]) # Se selecciona el individuo con mejor fitness
    bestChromosome = bestFit[0] #Se guarda el mejor individuo

    return bestChromosome

def crossover(parent1, parent2):
    """Se ejecuta la cruza con el método de 'Cruza de un Punto' entre los padres"""
    if random.random() < CROSSOVER_ODDS:
        point = random.randint(1, nAmpels-1) #Se selecciona el punto de cruza
        child1 = parent1[:point] + parent2[point:] # De la cruza se genera el hijo 1
        child2 = parent2[:point] + parent1[point:] # De la cruza se genera el hijo 2

    else:
        child1 = parent1
        child2 = parent2

    return child1, child2

def mutation(chromosome):
    """Se ejecuta la mutación mediante 'Mutación Uniforme' de un gen"""
    if random.random() < MUTATION_ODDS:
        gen = random.randint(1, nAmpels-1)
        chromosome[gen] = random.randint(MIN_TIME, MAX_TIME)

    return chromosome

In [5]:
def ampelOptimizationGA(showSimulation=False):
    """Obtiene los mejores tiempos en un sistema de semáforos para optimizar los tiempos de espera en intersecciones"""
    population = generatePopulation()
    vehicleRates = [random.random() for _ in range(nAmpels)]
    
    for generation in range(GENERATIONS):
        next_generation = []
        #Obtener la aptitud de cada individuo:
        cFinesess = [calculateFitness(chromosome, vehicleRates) for chromosome in population]

        while len(next_generation) < POPULATION_SIZE:
            #Se aplica selección:
            parent1 = selection(population, cFinesess)
            parent2 = selection(population, cFinesess)

            #Se aplica la cruza:
            child1, child2 = crossover(parent1, parent2)

            #Se aplica la mutación:
            child1 = mutation(child1)
            child2 = mutation(child2)

            next_generation.append(child1) # Se pasa el primer hijo a la siguiente generación

            if len(next_generation) < POPULATION_SIZE: # Sí no se ha revasado el tamaño de población
                next_generation.append(child2) # Se agrega también el segundo hijo

        # Se reemplaza la población con la nueva generación:
        population = next_generation

        #Mejores parámetros de la generación:
        bestFitness = max(cFinesess)
        bestChromosome = population[cFinesess.index(bestFitness)]
        print(f"\nGeneración No. {generation}: Mejor aptitud = {bestFitness} | Mejor individuo = {bestChromosome}")

        #Simular con el mejor cromosoma de la generación:
        ampelDict['cluster_1395059763_1658290098'][0] = bestChromosome
        projectFolder = infoSUMO['projectFolder']
        netFile = infoSUMO['netFile']
        sumoCfgFile = infoSUMO['sumoCfgFile']

        print(f"Comenzando simulación con el mejor individuo: ")
        simulation = SUMO(ampelDict, projectFolder, netFile, sumoCfgFile, SIMULATION_TIME, showSimulation)
        vehicleRates = simulation.simulateConditions(900)['cluster_1395059763_1658290098']
        print(f"Tasa de vehículos: {vehicleRates}")

        # Se reemplaza la población con la nueva generación:

    return bestChromosome

def simulateBest(bestSolution):
    """Simular con las mejores condiciones del algoritmo en la GUI de SUMO"""
    projectFolder = infoSUMO['projectFolder']
    netFile = infoSUMO['netFile']
    sumoCfgFile = infoSUMO['sumoCfgFile']

    ampelDict['cluster_1395059763_1658290098'][0] = bestSolution
    print("\n*************** Iniciando simulación con la mejor solución del algoritmo genético ***************")
    simulation = SUMO(ampelDict, projectFolder, netFile, sumoCfgFile, SIMULATION_TIME, showSimulation=True)
    simulation.simulateConditions()

In [6]:
bestSolution = ampelOptimizationGA() # Pasar True como argumento para ver cada simulación (cada generación)
print(f"Mejor solución {bestSolution}")
simulateBest(bestSolution)


Generación No. 0: Mejor aptitud = 165.33890664366876 | Mejor individuo = [87, 109, 113]
Comenzando simulación con el mejor individuo: 
Fase rrrrGGGGrrrrrrGGGGrr del semáforo cluster_1395059763_1658290098 modificada a 87 segundos.
Fase GGGGrrrrrrGGGGrrrrrr del semáforo cluster_1395059763_1658290098 modificada a 109 segundos.
Fase rrrrrrrrGGrrrrrrrrGG del semáforo cluster_1395059763_1658290098 modificada a 113 segundos.

 **************** Simulación Completada ******************
La tasa del semáforo 0 es de: 0.37777777777777777
La tasa del semáforo 1 es de: 0.11555555555555555
La tasa del semáforo 2 es de: 0.11222222222222222

 ************** Tasa de vehículos obtenida ****************
Tasa de vehículos: [0.37777777777777777, 0.11555555555555555, 0.11222222222222222]

Generación No. 1: Mejor aptitud = 59.41444444444444 | Mejor individuo = [87, 120, 113]
Comenzando simulación con el mejor individuo: 
Fase rrrrGGGGrrrrrrGGGGrr del semáforo cluster_1395059763_1658290098 modificada a 87 seg